In [1]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [3]:
'''
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
'''
    

'\nlinks = [\'https://github.com/poloclub/cnn-explainer\']\ngithub = \'https://api.github.com/\'\ngituser = \'https://github-user.now.sh/?username=\'\n    \nfor l in links:\n    repo_split = l.split(\'/\')\n    owner, repo = repo_split[-2], repo_split[-1]\n    print(\'Requestiong: \',owner, repo)\n    resp = rest_get(f\'{github}repos/{owner}/{repo}/contributors\')\n    for user in resp.json():\n        print(f\'user "{user["login"]}" has {user["contributions"]} contributions\')\n        user_info = rest_get(gituser+user["login"])\n        print(f"Name: {user_info[\'name\']}, Description: {user_info[\'description\']}")\n'

### Saving author contributions to txt file

In [4]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [5]:

repos = []
cursor.execute("SELECT DISTINCT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [6]:
len(repos)

11266

In [7]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [8]:
# Remove repos we already saved into file
with open(filename) as fp:
    line = fp.readline().split()
    while line:
        current_repo = line[0]
        if current_repo in repos:
            repos.remove(current_repo)
        line = fp.readline().split()
len(repos)

9041

In [9]:
repos.index('https://github.com/facebook/fb.resnet.torch')

ValueError: 'https://github.com/facebook/fb.resnet.torch' is not in list

In [10]:
missed = []

In [11]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Firefox()

for repo in repos[901:1000]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

[WDM] - Looking for [chromedriver 81.0.4044.138 linux64] driver in cache 
[WDM] - File found in cache by path [/home/kanter/.wdm/drivers/chromedriver/81.0.4044.138/linux64/chromedriver]


https://github.com/podgorskiy/StyleGan
https://github.com/podgorskiy
name: Stanislav Pidhorskyi, bio: PhD student/Research Assistant at WVU
Computer Vision, ML/AI, Computer graphics.
Interned at AWS Rekognition.
Ex gamedev, worked at Gameloft
User podgorskiy: commits 90 , changed lines + 6,999 - 3,116

https://github.com/r06922019/butt_lion_paper_notes
https://github.com/r06922019
name: , bio: 
User r06922019: commits 2 , changed lines + 9 - 7
https://github.com/jack71728
name: , bio: 
User jack71728: commits 1 , changed lines + 37 - 26

https://github.com/naka-lab/Serket
https://github.com/r-kuniyasu
name: , bio: 
User r-kuniyasu: commits 20 , changed lines + 660 - 514
https://github.com/naka-tomo
name: なかとも, bio: ここでは私が研究や勉強のために作成した資料やプログラムを公開しています．
User naka-tomo: commits 5 , changed lines + 58 - 52

https://github.com/hope-yao/robust_attention_cifar
https://github.com/hope-yao
name: , bio: 
User hope-yao: commits 1 , changed lines + 1,796 - 0

https://github.com/ruixif/Unet_keras
h

https://github.com/ChangweiZhang
name: Changwei Zhang, bio: iOS&Windows Developer
Working at developing HoloLens AR/MR apps
User ChangweiZhang: commits 44 , changed lines + 6,664 - 4,367

https://github.com/meet-minimalist/A-Neural-Algorithm-of-Artistic-Style-Paper-Implementation
https://github.com/meet-minimalist
name: Meet Patel, bio: Enabling machines to see since 2016.
User meet-minimalist: commits 20 , changed lines + 2,429 - 1,001

https://github.com/a-martyn/unet
https://github.com/a-martyn
name: , bio: 
User a-martyn: commits 40 , changed lines + 13,527 - 10,152

https://github.com/stanleycelestin1/AirsimDetectron
https://github.com/stanleycelestin1
name: Stanley Celestin, bio: Ayisien mwen ye. AI, ML, & Robotics.
User stanleycelestin1: commits 12 , changed lines + 36,188 - 1,544

https://github.com/jiajunhua/CycleGAN
https://github.com/junyanz
name: Jun-Yan Zhu, bio: Understanding and creating pixels.
User junyanz: commits 8 , changed lines + 20 - 15
https://github.com/taesung

name: , bio: Jenkins build machine.
User syglassbuild: commits 1 , changed lines + 2 - 0

https://github.com/vtddggg/BilinearGAN_for_LBIE
https://github.com/vtddggg
name: , bio: Student
User vtddggg: commits 4 , changed lines + 1,050 - 3

https://github.com/dobriban/Dist
https://github.com/dobriban
name: Edgar Dobriban, bio: Assistant Professor of Statistics. interests in stats and ML. GitHub has course notes, software, and code to reproduce papers.
User dobriban: commits 2 , changed lines + 1,653 - 1

https://github.com/bharatsush/object_detection
NOT FOUND/LOADED

https://github.com/danielwo/CarND-Traffic-Sign-Classifier-Project
https://github.com/andrewpaster
name: Andrew, bio: 
User andrewpaster: commits 34 , changed lines + 632 - 464
https://github.com/domluna
name: Dominique Luna, bio: magnificent stallion
User domluna: commits 33 , changed lines + 2,484 - 2,106
https://github.com/danielwo
name: Daniel Wood, bio: 
User danielwo: commits 28 , changed lines + 18,128 - 1,333
https:/

name: Antonia Tsili, bio: 
User sanantoniochili: commits 5 , changed lines + 731 - 1

https://github.com/UKPLab/emnlp2018-april
NOT FOUND/LOADED

https://github.com/vyq/behavioral-cloning
NOT FOUND/LOADED

https://github.com/oylz/FDSSTTest
https://github.com/oylz
name: xuyouzheng, bio: focus on computer vision.
User oylz: commits 2 , changed lines + 4,850 - 0

https://github.com/etttttte/mayfest2018
https://github.com/etttttte
name: etttttte, bio: eishii [at] connect.ust.hk
User etttttte: commits 71 , changed lines + 1,932 - 597

https://github.com/amarasovic/neural-abstract-anaphora
https://github.com/amarasovic
name: Ana Marasovic, bio: Postdoc at @allenai
User amarasovic: commits 4 , changed lines + 117 - 10

https://github.com/pravn/wasserstein_autoencoders
https://github.com/pravn
name: Praveen Narayanan, bio: Computational Scientist. 

I dabble in generative modeling. 
Most recent work has been on speech synthesis, domain adaptation.
User pravn: commits 7 , changed lines + 1,666 

In [13]:
driver = webdriver.Firefox() # 

for repo in repos[6701:6900]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/mathiasunberath/DeepDRR
https://github.com/mathiasunberath
name: Mathias, bio: 
User mathiasunberath: commits 13 , changed lines + 778 - 20
https://github.com/gaocong13
name: Cong Gao, bio: 
User gaocong13: commits 7 , changed lines + 427 - 6
https://github.com/Jan-Nico
name: Jan-Nico Zaech, bio: 
User Jan-Nico: commits 5 , changed lines + 20,136 - 8

https://github.com/carlren/gSLICr
NOT FOUND/LOADED

https://github.com/root-master/pose-anomaly-detection
NOT FOUND/LOADED

https://github.com/VLOGroup/tensorflow-icg
NOT FOUND/LOADED

https://github.com/YovaKem/generalized-procrustes-MUSE
NOT FOUND/LOADED

https://github.com/chl218/DCNN-on-FPGA
https://github.com/chl218
name: Chao Li (Jack), bio: 
User chl218: commits 22 , changed lines + 3,859,709 - 3,833,438

https://github.com/chrishokamp/constrained_decoding
https://github.com/chrishokamp
name: Chris Hokamp, bio: 
User chrishokamp: commits 107 , changed lines + 24,956 - 6,722
https://github.com/laeubli
name: Samuel

NOT FOUND/LOADED

https://github.com/panushri25/emrQA
https://github.com/panushri25
name: Anusri Pampari, bio: PhD student, Computer Science Department, Stanford University
User panushri25: commits 140 , changed lines + 265,551 - 258,413

https://github.com/craffel/mad
https://github.com/craffel
name: Colin Raffel, bio: 
User craffel: commits 9 , changed lines + 961 - 83

https://github.com/Mehran-k/RelNN
NOT FOUND/LOADED

https://github.com/JuneFeng/RelationClassification-RL
NOT FOUND/LOADED

https://github.com/dagrate/gan_network
NOT FOUND/LOADED

https://github.com/bpkindo/bayesqart
NOT FOUND/LOADED

https://github.com/AllenCellModeling/torch_integrated_cell
NOT FOUND/LOADED

https://github.com/huanzhang12/CLEVER
https://github.com/huanzhang12
name: Huan Zhang, bio: 
User huanzhang12: commits 27 , changed lines + 6,793 - 278
https://github.com/lilyweng
name: , bio: 
User lilyweng: commits 13 , changed lines + 941 - 169

https://github.com/andrecianflone/vector_quantization
NOT FOUND

https://github.com/yaoyao-liu
name: Yaoyao Liu, bio: 
User yaoyao-liu: commits 379 , changed lines + 9,262 - 7,936

https://github.com/shikorab/SceneGraph
https://github.com/shikorab
name: , bio: 
User shikorab: commits 8 , changed lines + 259 - 88
https://github.com/roeiherz
name: roei_herzig, bio: CS Ph.D. student at Tel Aviv University. Algorithm Researcher, R&D at Nexar & Trax. Studied MSc (CS), BSc (CS) and BSc (Physics) at TAU.
User roeiherz: commits 8 , changed lines + 2,976 - 101

https://github.com/OfekCohen1/Style-On-3D-Video
NOT FOUND/LOADED

https://github.com/mandarin4452/StyleTune
NOT FOUND/LOADED

https://github.com/ehsanik/SeGAN
https://github.com/ehsanik
name: Kiana Ehsani, bio: Phd student in Computer vision
User ehsanik: commits 4 , changed lines + 2 - 6

https://github.com/w510056105/DeepLearning
NOT FOUND/LOADED

https://github.com/HideakiImamura/MinimaxErrorRate
NOT FOUND/LOADED

https://github.com/ghaniskn/GCorp-Darknet
NOT FOUND/LOADED

https://github.com/episod

NOT FOUND/LOADED

https://github.com/mcordier/Papers-Presentation-ML-Reading-Group-
NOT FOUND/LOADED

https://github.com/hysts/pytorch_resnet_preact
NOT FOUND/LOADED

https://github.com/sirdr/TRPO.jl
https://github.com/sirdr
name: Loren AC, bio: Co-founder @ Melonfrost | M.S. Mechanical Engineering @ Stanford University
User sirdr: commits 99 , changed lines + 1,320 - 785
https://github.com/jacobmh1177
name: , bio: 
User jacobmh1177: commits 12 , changed lines + 910 - 109

https://github.com/DiaoXY/CSRnet
https://github.com/DiaoXY
name: diaoxiaoyu, bio: Want free bird
User DiaoXY: commits 1 , changed lines + 1,903 - 0

https://github.com/pariajm/emnlp2018
NOT FOUND/LOADED

https://github.com/cgnorthcutt/forum-diversification
NOT FOUND/LOADED

https://github.com/lawy623/LSTM_Pose_Machines
https://github.com/lawy623
name: lawy623, bio: Computer Vision, Machine Learning
User lawy623: commits 14 , changed lines + 163,348 - 20

https://github.com/JunYeopLee/capsule-networks
NOT FOUND/LOADED

name: Vatsal Goel, bio: A Deep Learning and Computer Vision enthusiast.
User vatsalg29: commits 1 , changed lines + 5 - 4
https://github.com/shubhamagarwal92
name: Shubham Agarwal, bio: PhD Student | Alana @ Alexa Prize 2018 | Pikabot @ VisDial 2018 |  Multimodal Dialog ||| 
Past: @AdobeResearch @gsoc | @XRCE | @IITDelhi
User shubhamagarwal92: commits 1 , changed lines + 1 - 1
https://github.com/rohrbach
name: Marcus Rohrbach, bio: 
User rohrbach: commits 1 , changed lines + 2 - 0
https://github.com/rmrao
name: Roshan Rao, bio: 
User rmrao: commits 1 , changed lines + 1 - 1
https://github.com/przemb
name: pb, bio: 
User przemb: commits 1 , changed lines + 1 - 0
https://github.com/omkar-fb
name: , bio: 
User omkar-fb: commits 1 , changed lines + 2 - 2
https://github.com/natviv
name: , bio: AI Researcher at Google
User natviv: commits 1 , changed lines + 8 - 0

https://github.com/rakhimovv/GenerativeLatentFlow
https://github.com/fursovia
name: Ivan Fursov, bio: Data Science student at Sk

In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Firefox()

for repo in repos[1001:1100]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

[WDM] - Looking for [chromedriver 81.0.4044.138 linux64] driver in cache 
[WDM] - File found in cache by path [/home/kanter/.wdm/drivers/chromedriver/81.0.4044.138/linux64/chromedriver]


https://github.com/lubagloukhova/food-101-keras
https://github.com/stratospark
name: Patrick Rodriguez, bio: 
User stratospark: commits 17 , changed lines + 9,663 - 1,889

https://github.com/Philip-Bachman/MatNets-NIPS
https://github.com/Philip-Bachman
name: Philip Bachman, bio: 
User Philip-Bachman: commits 2 , changed lines + 4 - 0

https://github.com/ColingPaper2018/DialogueAct-Tagger
https://github.com/alecervi
name: Alessandra Cervone, bio: 
User alecervi: commits 8 , changed lines + 17 - 12
https://github.com/ColingPaper2018
name: , bio: 
User ColingPaper2018: commits 3 , changed lines + 60 - 8

https://github.com/hexiangnan/attentional_factorization_machine
https://github.com/hexiangnan
name: He Xiangnan, bio: Research Fellow with School of Computing, National University of Singapore.
User hexiangnan: commits 10 , changed lines + 1,174 - 1,094
https://github.com/tonyfd
name: Ye Hao, bio: 
User tonyfd: commits 7 , changed lines + 2,296,512 - 33

https://github.com/jacquelinelala/

NOT FOUND/LOADED

https://github.com/adharshmurali/Decoding-Image-Classifier
https://github.com/adharshmurali
name: Adharsh Murali, bio: A computer and Data Science Enthusiast Interested in working in a complex challenging situation
User adharshmurali: commits 13 , changed lines + 672 - 17

https://github.com/weiylu/NLP
https://github.com/weiylu
name: , bio: 
User weiylu: commits 9 , changed lines + 109 - 50

https://github.com/guy-oren/DIRT-OST
https://github.com/hytseng0509
name: Hung-Yu Tseng, bio: PhD student @ UC Merced Vision and Learning Lab
User hytseng0509: commits 65 , changed lines + 3,079 - 1,359
https://github.com/HsinYingLee
name: Hsin-Ying Lee, bio: PhD student at UC Merced
User HsinYingLee: commits 20 , changed lines + 713 - 631
https://github.com/guy-oren
name: , bio: 
User guy-oren: commits 3 , changed lines + 344 - 172
https://github.com/nakosung
name: Nako Sung, bio: Naver Clova AI, former gamedev
(Author of Unreal.js)
User nakosung: commits 1 , changed lines + 1 - 

name: , bio: 
User alexanderrichard: commits 2 , changed lines + 116 - 115

https://github.com/tangzhenyu/tensorflow_tutorial
https://github.com/tangzhenyu
name: Alan Tang, bio: Interested in Machine Learning, Data Mining, Operation Research, Reinforcement Learning
User tangzhenyu: commits 19 , changed lines + 1,054,178 - 3

https://github.com/serinachang5/contextifier
https://github.com/serinachang5
name: Serina Chang, bio: 
User serinachang5: commits 6 , changed lines + 28 - 16
https://github.com/aethos
name: Ethan Adams, bio: Software Engineer at Asana.
BA in CS from Columbia University '18. Musician. Former TA.
User aethos: commits 1 , changed lines + 36 - 30
https://github.com/RuiqiZhong
name: , bio: 
User RuiqiZhong: commits 1 , changed lines + 1,129,594 - 0

https://github.com/yjhong89/Supplement-Pages
https://github.com/yjhong89
name: , bio: 
User yjhong89: commits 29 , changed lines + 160 - 71

https://github.com/hangyav/biadapt
https://github.com/hangyav
name: Hangya Viktor, 

KeyboardInterrupt: 

In [13]:
driver = webdriver.Firefox() # 

for repo in repos[7701:7800]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/Linxuxin/Pytorch-UNet-mine
NOT FOUND/LOADED

https://github.com/aonotas/deep-crf
https://github.com/aonotas
name: Motoki Sato, bio: Engineer at Preferred Networks. I'm interested in NLP and Deep Learning. #nlproc. (NAIST : Matsumoto Lab).
User aonotas: commits 116 , changed lines + 2,564 - 766
https://github.com/massongit
name: SUZUKI Masaya, bio: Data Engineer, Data Analyst or Software Engineer
User massongit: commits 48 , changed lines + 753 - 277
https://github.com/unnonouno
name: Yuya Unno, bio: Natural Language Processing, Deep Learning. Developer of Chainer and CuPy.
User unnonouno: commits 1 , changed lines + 1 - 1

https://github.com/Stick-To/RefineDet-tensorflow
NOT FOUND/LOADED

https://github.com/teakkkz/imageSR
NOT FOUND/LOADED

https://github.com/gurucharanmk/PyTorch_CosineAnnealingWithRestartsLR
NOT FOUND/LOADED

https://github.com/li012589/NeuralRG
NOT FOUND/LOADED

https://github.com/nguyenvo09/CombatingFakeNews
NOT FOUND/LOADED

https://github.com/ra

Exception: GET https://github.com/msftgits 502

In [ ]:
len(missed)

### Getting list of repositories and authors from the file

In [10]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [11]:
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

2079


In [12]:
author_dict['https://github.com/google/tf_mesh_renderer']

['Kyle Genova',
 'Forrester Cole',
 'Aaron Maschinot',
 'Aaron Sarna',
 'Daniel Vlasic',
 'William T. Freeman']

In [13]:
filename_mini = 'scraped_data_mini.txt' # use full version later

In [14]:
paper_data = {}

with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            current_repo = line[0]
            contributors = [line[1:]]                    

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

KeyError: 'https://github.com/benedekrozemberczki/karateclub'

In [15]:
len(paper_data)

212

### Creating graph

In [16]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF

In [ ]:
g = Graph()


### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master